In [5]:
import tensorflow as tf
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num CPUs Available:  1
Num GPUs Available:  1


In [6]:
import pandas as pd

digits_training_data = pd.read_csv('./train.csv')

# X is all the features I will be training the dataset on 
X = digits_training_data.copy()
# y is the prediction target
# pop label from the X and at the same time set the target
y = X.pop('label')

# X.head()
# y.head()

In [7]:
from sklearn.model_selection import train_test_split

# use part of the data from training and validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, train_size=0.75)

In [14]:
from tensorflow import keras
from tensorflow.keras import layers

# input_shape is the shape of the input data
model = keras.Sequential([
    layers.BatchNormalization(input_shape=(784,)),
    layers.Dense(32, activation='relu'),

    layers.BatchNormalization(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),

    layers.BatchNormalization(),
    layers.Dense(16, activation='relu'),

    layers.Dense(10, activation='softmax'),
])

In [19]:
# min_delta is the minimum change in the monitored quantity to qualify as an improvement, which is used for early stopping.
# patience is the number of epochs with no improvement after which training will be stopped.

early_stopping = keras.callbacks.EarlyStopping(
    patience=20,
    min_delta=0.001,
    restore_best_weights=True,
)

optimizer = keras.optimizers.Adam(lr=0.001)
# optimizer is the algorithm used to update the weights of the model.
# loss is the loss function to be minimized by the model.
model.compile(
    optimizer=optimizer,
    loss='mae',
    metrics=['accuracy']
)

# batch size is the number of samples (rows) that will be processed at a time
# epochs is the number of times the entire dataset will be processed
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=8,
    epochs=1000,
    callbacks=[early_stopping],
)

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()))

C:\Users\laimo\anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/1000
3938/3938 [==============================] - 18s 4ms/step - loss: 4.3764 - accuracy: 0.0983 - val_loss: 4.3763 - val_accuracy: 0.1066
Epoch 2/1000
1304/3938 [========>.....................] - ETA: 11s - loss: 4.3923 - accuracy: 0.0988

KeyboardInterrupt: 

In [1]:
digits_training_data = pd.read_csv('./test.csv')
test_X = digits_training_data.copy()

prediction = model.predict(test_X)
# output = pd.DataFrame({'imageId': digits_training_data.Id,
#                        'Label': prediction})
# output.to_csv('submission.csv', index=False)
# X.head()
# y.head()


submissions=pd.DataFrame({"ImageId": list(range(1,len(prediction)+1)),
                         "Label": prediction})
submissions.to_csv("DR2.csv", index=False, header=True)

NameError: name 'pd' is not defined